# SIT320 Advanced Algorithms
## Module 12 - MDP and Reinforcement Learning

---

<img src="uml-class-diagram.png" width="1000" height="800">

---

### Board Class

In [ ]:
from abc import ABC, abstractmethod

class Board(ABC):
    def __init__(self, boardDimensions):
        self.boardDimensions = boardDimensions

    @abstractmethod
    def createBoard(self):
        pass

    @abstractmethod
    def printBoard(self):
        pass

    @abstractmethod
    def setCellState(self, position, state):
        pass

    @abstractmethod
    def getCellState(self, position):
        pass

    @abstractmethod
    def getBoardState(self):
        pass

    @abstractmethod
    def getBoardDimensions(self):
        pass

    @abstractmethod
    def spaceIsFree(self, position):
        pass

    

In [ ]:
import itertools

class TicTacToeBoard(Board):
    def __init__(self, boardDimensions):
        """Create a board of dimensions boardDimensions x boardDimensions
        Args: boardDimensions (int): the dimensions of the board
        """
        self.boardDimensions = boardDimensions
        self.createBoard()


    def createBoard(self):
        """Create a board of dimensions boardDimensions x boardDimensions"""
        self.boardState = {i+1: ' ' for i in range(self.getBoardDimensions()**2)}

    def printBoard(self):
        for i in range(self.boardDimensions):
            row = [self.boardState[i*self.boardDimensions+j+1] for j in range(self.boardDimensions)]
            print('|'.join(row))
            if i < self.boardDimensions-1:
                print('-'*(self.boardDimensions*2-1))
        print('\n')

    def setCellState(self, position, state):
        """Set the state of a cell on the board
        Args:
            position (int): the position of the cell
            state (str): the state of the cell
        """
        self.boardState[position] = state

    def getCellState(self, position):
        """Get the state of a cell on the board
        Args:
            position (int): the position of the cell
        Returns: the state of the cell
        """
        return self.boardState[position]

    def getBoardState(self):
        """Get the state of the board
        Returns: A dictionary with keys 1 to boardDimensions**2 and values 'X', 'O' or ' '
        """
        return self.boardState

    def getBoardDimensions(self):
        """Get the dimensions of the board
        Returns: An integer representing the dimensions of the board 
        """
        return self.boardDimensions

    def getActions(self, state):
        """Get all valid actions for a given state."""
        return [
            i
            for i in range(1, self.boardDimensions**2 + 1)
            if state.getCellState(i) == ' '
        ]
    
    def spaceIsFree(self, position):
        if self.boardState[position] == ' ':
            return True

---

### Game Logic

In [ ]:
from abc import ABC, abstractmethod

class GameLogic():
    def __init__(self, boardGame):
        self.boardGame = boardGame

    @abstractmethod
    def chkForkWin(self):
        pass

    @abstractmethod
    def chkForDraw(self):
        pass

    @abstractmethod
    def chkMarkForWin(self, letter):
        pass

In [ ]:
class TicTacToeGameLogic(GameLogic):
    def __init__(self, boardGame):
        super().__init__(boardGame)
        """Create a game logic object for the board game
        Args: boardGame (Board): the board game. Must be a subclass of Board
        """

    def chkForDraw(self, state=None):
        """Check if the game is a draw.
        Returns: bool: True if the game is a draw, False otherwise.
        """
        if state is not None:
            boardState = state.getBoardState()
        else:
            boardState = self.boardGame.getBoardState()
        return all(boardState[key] != ' ' for key in boardState.keys())

    def chkForWin(self, state=None):
        """Check if any player has won.
        Returns: bool: True if any player has won, False otherwise.
        """
        if state is not None:
            boardState = state.getBoardState()
            boardDimensions = state.getBoardDimensions()
        else:
            boardState = self.boardGame.getBoardState()
            boardDimensions = self.boardGame.getBoardDimensions()
        for i in range(boardDimensions):
            row = [boardState[i*boardDimensions+j+1] for j in range(boardDimensions)]
            if len(set(row)) == 1 and row[0] != ' ':
                return True
        for i in range(boardDimensions):
            column = [boardState[j*boardDimensions+i+1] for j in range(boardDimensions)]
            if len(set(column)) == 1 and column[0] != ' ':
                return True
        diagonal1 = [boardState[i*boardDimensions+i+1] for i in range(boardDimensions)]
        diagonal2 = [boardState[i*boardDimensions+(boardDimensions-i-1)+1] for i in range(boardDimensions)]
        if len(set(diagonal1)) == 1 and diagonal1[0] != ' ':
            return True
        return len(set(diagonal2)) == 1 and diagonal2[0] != ' '

    def chkMarkForWin(self, letter, state=None):
        """Check if the player with the specified letter has won.
        Args: letter (str): Letter of the player to check for win.
        Returns: bool: True if the player with the specified letter has won, False otherwise.
        """
        if state is not None:
            boardState = state.getBoardState()
            boardDimensions = state.getBoardDimensions()
        else:
            boardState = self.boardGame.getBoardState()
            boardDimensions = self.boardGame.getBoardDimensions()
        # check rows
        for i in range(boardDimensions):
            row = [boardState[i*boardDimensions+j+1] for j in range(boardDimensions)]
            if len(set(row)) == 1 and row[0] == letter:
                return True
        for i in range(boardDimensions):
            column = [boardState[j*boardDimensions+i+1] for j in range(boardDimensions)]
            if len(set(column)) == 1 and column[0] == letter:
                return True
        diagonal1 = [boardState[i*boardDimensions+i+1] for i in range(boardDimensions)]
        diagonal2 = [boardState[i*boardDimensions+(boardDimensions-i-1)+1] for i in range(boardDimensions)]
        if len(set(diagonal1)) == 1 and diagonal1[0] == letter:
            return True
        return len(set(diagonal2)) == 1 and diagonal2[0] == letter

---

### Player Class

In [ ]:
from abc import ABC, abstractmethod

class Player(ABC):
    def __init__(self, letter, algorithm):
        self.letter = letter
        self.algorithm = algorithm
        """Create a player object
        Args: letter (str): the letter of the player. Must be 'X' or 'O'
        Args: algorithm (Algorithm): the algorithm used by the player. Must be a subclass of Algorithm
        """


    @abstractmethod
    def makeMove(self, boardGame):
        pass

In [ ]:
class HumanPlayer(Player):
    def __init__(self, letter, algorithm):
        self.letter = letter
        self.algorithm = algorithm
    
    # function for player to choose a position
    def makeMove(self, boardGame):
        """Make a move by asking for input from the user.
        Args: boardGame (Board): The board game object.
        If the position is not free, ask for another position.
        If the position is free, set the cell state to the player's letter.
        """
        position = self.algorithm.bestMove(boardGame, self.letter)
        boardGame.setCellState(position, self.letter)

In [ ]:
class ComputerPlayer(Player):
    def __init__(self, letter, algorithm):
        self.letter = letter
        self.algorithm = algorithm
    
    def makeMove(self, boardGame):
        """Make a move by using the algorithm to find the best move.
        Args: boardGame (Board): The board game object.
        Set the cell state to the player's letter.
        """
        print('Computer is thinking...')
        position = self.algorithm.bestMove(boardGame, self.letter)
        print(f"Computer chose position: {position}")
        boardGame.setCellState(position, self.letter)

---

### Algorithms

In [ ]:
class Algorithm(ABC):
    def __init__(self, boardGame):
        self.boardGame = boardGame
        """Create an algorithm object for the board game
        Args: boardGame (Board): the board game. Must be a subclass of Board
        """
    
    @abstractmethod
    def bestMove(self, boardGame, letter):
        pass


### MiniMax

In [ ]:
class Minimax(Algorithm):
    def __init__(self, boardGame):
        super().__init__(boardGame)

    def bestMove(self, boardGame, letter):
        """Find the best move for the computer player.
        Args: boardGame (Board): The board game object
        Args: letter (str): The letter of the computer player.
        Returns: int: The position of the best move."""
        boardState = boardGame.getBoardState()
        bestScore = -1000
        bestMove = 0
        for key in boardState.keys():
            if boardState[key] == ' ':
                boardState[key] = letter
                score = self.minimax(boardState, 0, False, letter)
                boardState[key] = ' '
                if score > bestScore:
                    bestScore = score
                    bestMove = key
        return bestMove

    def minimax(self, boardState, depth, isMaximizing, letter, maxDepth=5):
        """Find the best score for the computer player.
        Args: boardState (dict): The board state.
        Args: depth (int): The depth of the tree.
        Args: isMaximizing (bool): Whether the player is maximizing or not.
        Args: letter (str): The letter of the computer player.
        Args: maxDepth (int): The maximum depth of the tree.
        Returns: int: The best score for the computer player."""
        
        gameLogic = TicTacToeGameLogic(self.boardGame)
        opponentLetter = 'O' if letter == 'X' else 'X'
        if(gameLogic.chkMarkForWin(letter)):
            return 1
        elif(gameLogic.chkMarkForWin(opponentLetter)):
            return -1
        elif(gameLogic.chkForDraw()):
            return 0
        elif depth >= maxDepth:
            return 0

        if isMaximizing:
            bestScore = -1000
            for key in boardState.keys():
                if boardState[key] == ' ':
                    boardState[key] = letter
                    score = self.minimax(boardState, depth + 1, False, letter)
                    boardState[key] = ' '
                    if score > bestScore:
                        bestScore = score
        else:
            bestScore = 1000
            for key in boardState.keys():
                if boardState[key] == ' ':
                    boardState[key] = opponentLetter
                    score = self.minimax(boardState, depth + 1, True, letter)
                    boardState[key] = ' '
                    if score < bestScore:
                        bestScore = score
        return bestScore

### Minimax with Alpha-Beta Pruning

In [ ]:
class MinimaxAlphaBeta(Algorithm):
    def __init__(self, boardGame):
        super().__init__(boardGame)

    def bestMove(self, boardGame, letter):
        boardState = boardGame.getBoardState()
        bestScore = -1000
        bestMove = 0
        for key in boardState.keys():
            if boardState[key] == ' ':
                boardState[key] = letter
                score = self.minimax(boardState, 0, False, letter)
                boardState[key] = ' '
                if score > bestScore:
                    bestScore = score
                    bestMove = key
        return bestMove

    def minimax(self, boardState, depth, isMaximizing, letter, alpha=-1000, beta=1000, maxDepth=5):
        """Find the best score for the computer player.
        Args: boardState (dict): The board state.
        Args: depth (int): The depth of the tree.
        Args: isMaximizing (bool): Whether the player is maximizing or not.
        Args: letter (str): The letter of the computer player.
        Args: alpha (int): The alpha value. 
        Args: beta (int): The beta value.
        Args: maxDepth (int): The maximum depth of the tree.
        Returns: int: The best score for the computer player."""
        gameLogic = TicTacToeGameLogic(self.boardGame)
        opponentLetter = 'O' if letter == 'X' else 'X'
        if(gameLogic.chkMarkForWin(letter)):
            return 1
        elif(gameLogic.chkMarkForWin(opponentLetter)):
            return -1
        elif(gameLogic.chkForDraw()):
            return 0
        elif depth >= maxDepth:
            return 0

        if isMaximizing:
            bestScore = -1000
            for key in boardState.keys():
                if boardState[key] == ' ':
                    boardState[key] = letter
                    score = self.minimax(boardState, depth+1, False, letter, alpha, beta)
                    boardState[key] = ' '
                    bestScore = max(score, bestScore)
                    alpha = max(alpha, score)
                    if beta <= alpha:
                        break
        else:
            bestScore = 1000
            for key in boardState.keys():
                if boardState[key] == ' ':
                    boardState[key] = opponentLetter
                    score = self.minimax(boardState, depth+1, True, letter, alpha, beta)
                    boardState[key] = ' '
                    bestScore = min(score, bestScore)
                    beta = min(beta, score)
                    if beta <= alpha:
                        break
        return bestScore

## Reinforcement Learning 

---


### Value Iteration
- These classes solve the tic-tac-toe game using the value iteration algorithm.
- We assume the tic-tac-toe game is an MDP with states, actions, transition probabilities, and rewards.
- States: the board state.
- Actions: the position to place the next move.
- Transition probabilities: 1.0 if the move is valid, 0.0 otherwise.
- Rewards: 1 if the move is a winning move, 0 otherwise.

In [ ]:
import random
import copy

class ValueIteration(Algorithm):
    def __init__(self, boardGame):
        super().__init__(boardGame)
        self.policy =  {}
        self.G = Graph()
        self.value_function = {}
        self.valueIteration()


    def bestMove(self, boardGame, letter):
    # selects best move from policy
        boardState = boardGame.getBoardState()
        boardStateTuple = tuple(boardState.values())
        

        if action := self.policy.get(boardStateTuple, None):
            print(f"Action found in policy: {action}")
            print(f"Action: {action}")
            return action
        else:
            print("No action found in policy for this state.")
            

    def valueIteration(self):
    # begins the value iteration process
    # calls initialize and converge
        game_logic = TicTacToeGameLogic(self.boardGame)
        best_policy = {}
        blankBoard = copy.deepcopy(self.boardGame)
        graph = self.initialize(self.G, blankBoard, game_logic)
        print("Graph initialized")
        self.value_function = {state: 0 for state in graph.nodes}
        if self.converge(game_logic):
            print("Converged")
        print(f"Value function: {self.value_function}")

    def initialize(self, graph, board, game_logic):
    # initializes the graph with all possible states and actions as nodes and edges
    # assigns a value to end states
        print("Initializing...")
        print(f"Start state: {board.getBoardState()}")
        graph.add_node(tuple(board.getBoardState().values()))
        queue = [(board, 'X')]  # Add the player to the queue
        while queue:
            state, current_player = queue.pop(0)
            for action in board.getActions(state):
                new_state = copy.deepcopy(state)
                new_state.setCellState(action, current_player)
                new_state_tuple = tuple(new_state.getBoardState().values())
                if new_state_tuple not in graph.nodes:
                    graph.add_node(new_state_tuple)
                    next_player = 'O' if current_player == 'X' else 'X'
                    queue.append((new_state, next_player))
                # reward is random float between 0 and 1
                graph.add_edge(tuple(state.getBoardState().values()), new_state_tuple, 0)
                if game_logic.chkForWin(new_state) and game_logic.chkMarkForWin('X', new_state):
                    graph.nodes[new_state_tuple].value = 1
                elif game_logic.chkForWin(new_state) and game_logic.chkMarkForWin('O', new_state):
                    graph.nodes[new_state_tuple].value = -1
                elif game_logic.chkForDraw(new_state):
                    graph.nodes[new_state_tuple].value = 0
                graph.update_edge_reward(tuple(state.getBoardState().values()), new_state_tuple)
        if not graph.nodes:
            raise ValueError("Graph is empty")
        return graph


    def converge(self, game_logic, gamma=0.9, epsilon=1e-4):
    # Initialize delta to a large value to enter the loop
        delta = float('inf')
        i = 0
        # Loop until the value function converges
        while delta > epsilon:
            delta = 0  # Reset delta for each iteration

            # Iterate through all states
            for curr_state in self.G.nodes.values():
                old_value = self.value_function.get(curr_state.state, 0)  # Get the old value function for the state
                best_action_value = self.calculateBestAction(curr_state.state, gamma)  # Calculate the best action value
                # print(f"Old value for state {curr_state.state}: {old_value}")
                # print(f"New value for state {curr_state.state}: {best_action_value}")

                # Update the value function
                self.value_function[curr_state.state] = best_action_value

                # Update the policy
                self.policy[curr_state.state] = self.calculateBestAction(curr_state.state, gamma, return_action=True)

                # Calculate the change in the value function for this state
                delta = max(delta, abs(old_value - best_action_value))
            i += 1
        print(f"Number of iterations: {i}")

        print("Value function converged.")


    def calculateBestAction(self, state_tuple, gamma, return_action=False):
    # Initialize variables
        max_value = float('-inf')  # Initialize to negative infinity
        best_action = None

        # Get the neighbors (possible next states) for the current state
        neighbors = self.G.get_neighbors(state_tuple)

        # If there are no neighbors, return 0 or None based on the flag
        if not neighbors:
            return 0 if not return_action else None

        # Iterate through each neighbor to find the best action
        for neighbor in neighbors:
            reward = self.G.get_reward(state_tuple, neighbor)  # Get the reward for transitioning to the neighbor
            value = reward + (gamma * self.value_function.get(neighbor, 0))  # Calculate the value of the action

            # Update max_value and best_action if this action is better
            if value > max_value:
                max_value = value
                best_action = self.new_X_position(state_tuple, neighbor) + 1  # Update the best action

        # Update the value function for the current state
        self.value_function[state_tuple] = max_value

    # Return either the best action's value or the best action itself based on the flag
        return max_value if not return_action else best_action
 # Index starts at 0

    def new_X_position(self, state_tuple, chosen_neighbor):
    # find the position of the X in the chosen neighbor
        return [
            i
            for i, (current, next) in enumerate(zip(state_tuple, chosen_neighbor))
            if current != next
        ][0]






def test_converge():
    # Initialize a ValueIteration object with a mock game board and logic
    game_factory = TicTacToeCreator(2)
    # Test that the value function and policy are empty before convergence
    testVI = ValueIteration(game_factory.board)
    # Test that the value function and policy are no longer empty
    

# test_converge()
# play_game()

In [ ]:
class Node:
    def __init__(self, state, value):
        self.state = state
        self.value = value

    def __str__(self):
        return f"State: {self.state}, Value: {self.value}"

        

class Graph:
    def __init__(self):
        self.nodes = {}
        self.edges = {}

    def add_node(self, state, value=0):
        node = Node(state, value)
        self.nodes[state] = node
        return node

    def add_edge(self, state1, state2, reward=0):
        if state1 not in self.nodes:
            self.add_node(state1)
        if state2 not in self.nodes:
            self.add_node(state2)
        if state1 not in self.edges:
            self.edges[state1] = {}
        
        # Use the value of the destination node as the reward
        reward = self.nodes[state2].value if self.nodes[state2].value != 0 else reward
        # print(f"Reward: {reward}")
        self.edges[state1][state2] = reward

    def update_edge_reward(self, state1, state2):
        if state1 in self.edges and state2 in self.edges[state1]:
            self.edges[state1][state2] = self.nodes[state2].value
            # print(f"Updated edge reward: {self.edges[state1][state2]}")



    def get_neighbors(self, state):
        return [] if state not in self.edges else list(self.edges[state].keys())

    def get_reward(self, state1, state2):
        if state1 not in self.edges or state2 not in self.edges[state1]:
            return 0
        # print(self.edges[state1][state2])
        return self.edges[state1][state2]

    def print_graph(self):
        for i, node in enumerate(self.nodes.values(), start=1):
            print(f"State {i} Node: {node}")
            neighbors = self.get_neighbors(node.state)
            for neighbor in neighbors:
                reward = self.get_reward(node.state, neighbor)
                print(f"  -> State {self.get_node_index(neighbor)} (Reward={reward})")

    def get_node_index(self, state):
        return list(self.nodes.keys()).index(state) + 1

    def count_winning_states(self):
        return sum(1 for node in self.nodes.values() if node.value == 1)
    
    def count_losing_states(self):
        return sum(1 for node in self.nodes.values() if node.value == -1)

    def count_draw_states(self):
        return sum(1 for node in self.nodes.values() if node.value == 0)


### Q-Learning

In [ ]:
class QLearning(Algorithm):
    def __init__(self, boardGame):
        super().__init__(boardGame)
    
    def bestMove(self, boardGame, letter):
        pass

---

### Algorithm for the human player

In [ ]:
class UserInput(Algorithm):
    def __init__(self, boardGame):
        super().__init__(boardGame)

    def bestMove(self, boardGame, letter):
        """Ask the user for input.
        Args: boardGame (Board): The board game object.
        Args: letter (str): The letter of the computer player.
        Returns: int: The position of the user's input."""
        while True:
            try:
                position = int(input("Please enter a position: "))
                if position < 1 or position > boardGame.getBoardDimensions()**2:
                    raise ValueError
                if boardGame.spaceIsFree(position):
                    return position
                else:
                    raise ValueError
            except ValueError:
                print("Invalid input!")

In [ ]:
class AbstractGameFactory(ABC):

    @abstractmethod
    def createGameLogic(self) -> GameLogic:
        pass

    @abstractmethod
    def createPlayer(self) -> Player:
        pass

    @abstractmethod
    def createAlgorithm(self) -> Algorithm:
        pass

In [ ]:
class TicTacToeCreator(AbstractGameFactory):
    def __init__(self, dimensions):
        self.board = TicTacToeBoard(dimensions)

    def createGameLogic(self) -> GameLogic:
        """Create a game logic object for the board game
        Returns: gameLogic (GameLogic): the game logic object. Must be a subclass of GameLogic
        """
        return TicTacToeGameLogic(self.board)

    def createPlayer(self, letter, isComputer, algorithm) -> Player:
        """Create a player object
        Args: letter (str): the letter of the player. Must be 'X' or 'O'
        Args: isComputer (bool): whether the player is a computer or not
        Args: algorithm (Algorithm): the algorithm used by the player. Must be a subclass of Algorithm
        Returns: player (Player): the player object. Must be a subclass of Player
        """
        if isComputer:
            return ComputerPlayer(letter, algorithm)
        return HumanPlayer(letter, algorithm)

    def createAlgorithm(self, algorithm) -> Algorithm:
        """Create an algorithm object for the board game
        Args: algorithm (int): the algorithm used by the player. Must be 1, 2, 3 or 4
        Returns: algorithm (Algorithm): the algorithm object. Must be a subclass of Algorithm
        """
        if algorithm == 1:
            return Minimax(self.board)
        elif algorithm == 2:
            return MinimaxAlphaBeta(self.board)
        elif algorithm == 3:
            return ValueIteration(self.board)
        elif algorithm == 4:
            return QLearning(self.board)
        elif algorithm == 5:
            return UserInput(self.board)

---

## Game

In [ ]:
import time

def prompt_user(prompt, options):
    """Prompt the user to choose an option.
    Args: prompt (str): The prompt to display to the user.
    Args: options (list): The list of options to display to the user.
    Returns: int: The option chosen by the user.
    """
    print(prompt)
    for i, option in enumerate(options):
        print(f"{i+1}. {option}")
    choice = int(input("Please choose an option: "))
    while choice < 1 or choice > len(options):
        print(f"Choice must be between 1 and {len(options)}!")
        choice = int(input("Please choose an option: "))
    return choice

def choose_game():
    """Prompt the user to choose a game.
    Returns: game_factory (AbstractGameFactory): The game factory for the game chosen by the user.
    """
    prompt = "Welcome to the Game Factory!\nPlease choose a game:"
    options = ["Tic Tac Toe", "Chess", "Backgammon"]
    choice = prompt_user(prompt, options)
    if choice == 1:
        dimensions = int(input("Please enter the board dimensions: "))
        return TicTacToeCreator(dimensions)
    elif choice == 2:
        return ChessCreator()
    elif choice == 3:
        return BackgammonCreator()

def choose_algorithm():
    """Prompt the user to choose an algorithm.
    Returns: int: The algorithm chosen by the user.
    """
    prompt = "Which Algorithm should player use?"
    options = ["Minimax", "Minimax with Alpha Beta Pruning", "Value Iteration", "Q-Learning", "User Input"]
    return prompt_user(prompt, options)

def create_player(game_factory, letter):
    """Create a player object
    Args: game_factory (AbstractGameFactory): The game factory for the game chosen by the user.
    Args: letter (str): the letter of the player. Must be 'X' or 'O'
    Returns: player (Player): the player object. Must be a subclass of Player
    """
    is_computer = input(f"Is player {letter} a computer? (Y/N): ")
    while is_computer not in ['Y', 'N']:
        print("Invalid input!")
        is_computer = input(f"Is player {letter} a computer? (Y/N): ")
    algorithm_choice = choose_algorithm()
    algorithm = game_factory.createAlgorithm(algorithm_choice)
    return game_factory.createPlayer(letter, is_computer == 'Y', algorithm)

def play_game():
    """Play the game."""
    game_factory = choose_game()
    game_logic = game_factory.createGameLogic()
    player_one = create_player(game_factory, 'X')
    player_two = create_player(game_factory, 'O')
    game_factory.board.printBoard()
    start = time.time()
    while not game_logic.chkForWin() and not game_logic.chkForDraw():
        player_one.makeMove(game_factory.board)
        game_factory.board.printBoard()
        if game_logic.chkForWin():
            print("Player", player_one.letter, "wins!")
            end = time.time()
            print("Time taken: ", end - start)
            break
        elif game_logic.chkForDraw():
            print("It's a draw!")
            end = time.time()
            print("Time taken: ", end - start)
            break
        player_two.makeMove(game_factory.board)
        game_factory.board.printBoard()
        if game_logic.chkForWin():
            print("Player", player_two.letter, "wins!")
            end = time.time()
            print("Time taken: ", end - start)
            break
        elif game_logic.chkForDraw():
            print("It's a draw!")
            end = time.time()
            print("Time taken: ", end - start)
            break

play_game()